**Import pacakages**

In [1]:
from sqlalchemy import create_engine
import pandas as pd

**Q1**: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. 

In [2]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
df = pd.read_sql_query('''SELECT name 
                          FROM Facilities 
                          WHERE membercost !=0''', engine)
print(df)


             name
0  Tennis Court 1
1  Tennis Court 2
2  Massage Room 1
3  Massage Room 2
4    Squash Court


**Q2**: How many facilities do not charge a fee to members?


In [3]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
df = pd.read_sql_query('''SELECT COUNT(*) as count 
                          FROM Facilities 
                          WHERE membercost ==0''', engine)
print(df)


   count
0      4


**Q3**: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [4]:
df = pd.read_sql_query('''SELECT facid, name, membercost, monthlymaintenance 
                          FROM Facilities 
                          WHERE membercost > 0 AND membercost < 0.2 * (monthlymaintenance)''', engine)
print(df)

   facid            name  membercost  monthlymaintenance
0      0  Tennis Court 1         5.0                 200
1      1  Tennis Court 2         5.0                 200
2      4  Massage Room 1         9.9                3000
3      5  Massage Room 2         9.9                3000
4      6    Squash Court         3.5                  80


**Q4**: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [5]:
df = pd.read_sql_query('''SELECT * 
                          FROM Facilities
                          WHERE facid IN (1, 5) ''', engine)
print(df)

   facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0         25           8000   
1      5  Massage Room 2         9.9         80           4000   

   monthlymaintenance  
0                 200  
1                3000  


**Q5**: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. 

In [6]:
df = pd.read_sql_query('''SELECT name, monthlymaintenance,
                            CASE WHEN monthlymaintenance <100 THEN 'cheap'
                            ELSE 'expensive'
                            END AS TYPE
                          FROM Facilities''', engine)
print(df)

              name  monthlymaintenance       TYPE
0   Tennis Court 1                 200  expensive
1   Tennis Court 2                 200  expensive
2  Badminton Court                  50      cheap
3     Table Tennis                  10      cheap
4   Massage Room 1                3000  expensive
5   Massage Room 2                3000  expensive
6     Squash Court                  80      cheap
7    Snooker Table                  15      cheap
8       Pool Table                  15      cheap


**Q6**: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. 

In [7]:
df = pd.read_sql_query('''SELECT firstname, surname, joindate
                          FROM Members
                          WHERE joindate = (
                          SELECT MAX(joindate)
                          FROM Members)''', engine)
print(df)

  firstname surname             joindate
0    Darren   Smith  2012-09-26 18:08:45


**Q7**: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [8]:
df = pd.read_sql_query('''SELECT DISTINCT m.firstname || " " || m.surname AS member, f.name AS court_name
                          FROM Bookings AS b
                          LEFT JOIN Members AS m ON m.memid = b.memid
                          LEFT JOIN Facilities AS f ON b.facid = f.facid
                          WHERE b.facid IN (0, 1) AND m.surname != 'GUEST'
                          ORDER BY court_name''', engine)
print(df)

               member      court_name
0         Tracy Smith  Tennis Court 1
1          Tim Rownam  Tennis Court 1
2     Janice Joplette  Tennis Court 1
3      Gerald Butters  Tennis Court 1
4        Burton Tracy  Tennis Court 1
5          Nancy Dare  Tennis Court 1
6          Tim Boothe  Tennis Court 1
7        Charles Owen  Tennis Court 1
8          Anne Baker  Tennis Court 1
9          Jack Smith  Tennis Court 1
10        David Jones  Tennis Court 1
11      Timothy Baker  Tennis Court 1
12       David Pinker  Tennis Court 1
13    Ponder Stibbons  Tennis Court 1
14        Joan Coplin  Tennis Court 1
15      Douglas Jones  Tennis Court 1
16   Ramnaresh Sarwin  Tennis Court 1
17     Jemima Farrell  Tennis Court 1
18      David Farrell  Tennis Court 1
19     Florence Bader  Tennis Court 1
20          John Hunt  Tennis Court 1
21    Matthew Genting  Tennis Court 1
22      Erica Crumpet  Tennis Court 1
23         Tim Rownam  Tennis Court 2
24       Darren Smith  Tennis Court 2
25    Janice

**Q8**: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [9]:
df = pd.read_sql_query('''SELECT f.name AS facility, m.firstname || " " || m.surname AS name,
                             CASE WHEN b.memid !=0 THEN b.slots*(f.membercost)
                             ELSE b.slots*(f.guestcost)
                             END AS cost
                          FROM Members AS m
                          LEFT JOIN Bookings AS b ON m.memid = b.memid
                          LEFT JOIN Facilities AS f ON f.facid = b.facid
                          WHERE b.starttime LIKE '2012-09-14 %'
                                AND CASE WHEN b.memid !=0 THEN b.slots *(f.membercost)
                                ELSE b.slots *(f.guestcost)
                                END >30
                          ORDER BY cost DESC''', engine)
print(df)

          facility            name   cost
0   Massage Room 2     GUEST GUEST  320.0
1   Massage Room 1     GUEST GUEST  160.0
2   Massage Room 1     GUEST GUEST  160.0
3   Massage Room 1     GUEST GUEST  160.0
4   Tennis Court 2     GUEST GUEST  150.0
5   Tennis Court 1     GUEST GUEST   75.0
6   Tennis Court 1     GUEST GUEST   75.0
7   Tennis Court 2     GUEST GUEST   75.0
8     Squash Court     GUEST GUEST   70.0
9   Massage Room 1  Jemima Farrell   39.6
10    Squash Court     GUEST GUEST   35.0
11    Squash Court     GUEST GUEST   35.0


**Q9**: This time, produce the same result as in Q8, but using a subquery.

In [10]:
df = pd.read_sql_query('''SELECT *
                          FROM (
                              SELECT f.name AS facility, m.firstname || " " || m.surname AS name,
                                 CASE WHEN b.memid !=0 THEN b.slots*(f.membercost)
                                  ELSE b.slots*(f.guestcost)
                                  END AS cost
                              FROM Members AS m
                              LEFT JOIN Bookings AS b ON m.memid = b.memid
                              LEFT JOIN Facilities AS f ON f.facid = b.facid
                              WHERE b.starttime LIKE '2012-09-14 %'
                              ) AS cost
                         WHERE cost >30
                         ORDER BY cost DESC''', engine)
print(df)

          facility            name   cost
0   Massage Room 2     GUEST GUEST  320.0
1   Massage Room 1     GUEST GUEST  160.0
2   Massage Room 1     GUEST GUEST  160.0
3   Massage Room 1     GUEST GUEST  160.0
4   Tennis Court 2     GUEST GUEST  150.0
5   Tennis Court 1     GUEST GUEST   75.0
6   Tennis Court 1     GUEST GUEST   75.0
7   Tennis Court 2     GUEST GUEST   75.0
8     Squash Court     GUEST GUEST   70.0
9   Massage Room 1  Jemima Farrell   39.6
10    Squash Court     GUEST GUEST   35.0
11    Squash Court     GUEST GUEST   35.0


**Q10**: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [11]:
df = pd.read_sql_query('''SELECT *
                          FROM
                            (SELECT f.name AS facility, SUM(CASE WHEN m.firstname = "GUEST" THEN slots*f.guestcost
                                                            ELSE slots*f.membercost
                                                            END) AS revenue
                            FROM Members AS m
                            INNER JOIN Bookings AS b
                            ON m.memid = b.memid
                            INNER JOIN Facilities AS f
                            ON b.facid = f.facid
                            GROUP BY f.name)
                        WHERE revenue < 1000
                        ORDER BY revenue DESC''', engine)
print(df)

        facility  revenue
0     Pool Table      270
1  Snooker Table      240
2   Table Tennis      180


**Q11**: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [67]:
df = pd.read_sql_query('''
                        SELECT r.surname || ', ' || r.firstname as member_name, m.surname || ', ' || m.firstname as recommender_name
                        FROM Members AS m
                        JOIN Members AS r ON m.memid==r.recommendedby
                        ORDER BY member_name
                       ''', engine)
print(df)

                 member_name    recommender_name
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8           Genting, Matthew     Butters, Gerald
9                 Hunt, John  Purview, Millicent
10              Jones, David    Joplette, Janice
11            Jones, Douglas        Jones, David
12          Joplette, Janice       Smith, Darren
13           Mackenzie, Anna       Smith, Darren
14             Owen, Charles       Smith, Darren
15             Pinker, David     Farrell, Jemima
16        Purview, Millicent        Smith, Tracy
17         Rumney, Henrietta    Genting, Matthew
18         Sarwin, Ramnaresh     Bader, Florence
19               Smi

**Q12**: Find the facilities with their usage by member, but not guests 

In [72]:
# usage = number of members who used 
df = pd.read_sql_query('''SELECT f.name, b.member_count
                          FROM
                              (SELECT facid, COUNT(*) as member_count
                              FROM Bookings
                              WHERE memid != 0
                              GROUP BY facid) AS b
                          JOIN Facilities AS f on f.facid == b.facid
                          ORDER BY member_count DESC
                          ''', engine)
print(df)

              name  member_count
0       Pool Table           783
1   Massage Room 1           421
2    Snooker Table           421
3     Table Tennis           385
4  Badminton Court           344
5   Tennis Court 1           308
6   Tennis Court 2           276
7     Squash Court           195
8   Massage Room 2            27


**Q13**: Find the facilities usage by month, but not guests

In [96]:
df = pd.read_sql_query('''SELECT f.name, b.member_count, b.Month
                          FROM
                              (SELECT facid, COUNT(*) as member_count, strftime('%m', starttime) AS Month
                              FROM Bookings
                              WHERE memid != 0
                              GROUP BY facid, Month) AS b
                          JOIN Facilities AS f on f.facid == b.facid
                          ORDER BY member_count DESC
                          ''', engine)
print(df)

               name  member_count Month
0        Pool Table           408    09
1        Pool Table           272    08
2     Snooker Table           199    09
3      Table Tennis           194    09
4    Massage Room 1           191    09
5   Badminton Court           161    09
6     Snooker Table           154    08
7    Massage Room 1           153    08
8      Table Tennis           143    08
9    Tennis Court 1           132    09
10  Badminton Court           132    08
11   Tennis Court 2           126    09
12   Tennis Court 1           111    08
13   Tennis Court 2           109    08
14       Pool Table           103    07
15     Squash Court            87    09
16     Squash Court            85    08
17   Massage Room 1            77    07
18    Snooker Table            68    07
19   Tennis Court 1            65    07
20  Badminton Court            51    07
21     Table Tennis            48    07
22   Tennis Court 2            41    07
23     Squash Court            23    07


In [95]:
df = pd.read_sql_query('''SELECT facid, COUNT(*) as member_count, MONTH(starttime) AS Month
                              FROM Bookings
                          ''', engine)
print(df)

OperationalError: (sqlite3.OperationalError) no such function: MONTH
[SQL: SELECT facid, COUNT(*) as member_count, MONTH(starttime) AS Month
                              FROM Bookings
                          ]
(Background on this error at: https://sqlalche.me/e/14/e3q8)